# Basic tests


In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerlib.span import Span
from spannerlib.session import Session,test_session

In [ ]:
# test assignment
test_session(
    """
        new Relation(int, int)
        x = 1
        y = 2
        z = y
        Relation(x, y)
        Relation(y, x)
        ?Relation(X, x)
    """,
    pd.DataFrame([[2]],columns=['X']),
)

adding relations for name='Relation' terms=[FreeVar(name='X'), 1]


'?Relation(X,x)'

,X
0,2


In [ ]:
# test copy table rule
test_session(
    """
        new B(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        A(X, Y) <- B(X, Y)
        ?A(X, Y)
    """,
    pd.DataFrame(
        [[1,1],[1,2],[2,3]],
        columns=['X','Y']
    )
)

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='Y')]


'?A(X,Y)'

,X,Y
0,1,1
1,1,2
2,2,3


In [ ]:
# test join 2 tables
test_session(
    """
        new B(int, int)
        new C(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        C(2, 2)
        C(1, 1)
        D(X, Y, Z) <- B(X, Y), C(Y, Z)
        ?D(X, Y, Z)
    """,
    pd.DataFrame(
        [[1,2,2],[1,1,1]],
        columns=['X','Y','Z']
    )
)

adding relations for name='D' terms=[FreeVar(name='X'), FreeVar(name='Y'), FreeVar(name='Z')]


'?D(X,Y,Z)'

,X,Y,Z
0,1,1,1
1,1,2,2


In [ ]:
# rel with same free var
test_session(
    """
        new B(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 2)
        A(X) <- B(X, X)
        ?A(X)
    """,
    pd.DataFrame(
        [[1],[2]],
        columns=['X']
    )
)

adding relations for name='A' terms=[FreeVar(name='X')]


'?A(X)'

,X
0,1
1,2


In [ ]:
# union with same vars
test_session(
    """
        new B(int, int)
        new C(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        C(2, 2)
        C(1, 1)

        A(X, Y) <- B(X, Y)
        A(X, Y) <- C(X, Y)
        ?A(X, Y)
    """,
    pd.DataFrame(
        [[1,1],[1,2],[2,2],[2,3]],
        columns=['X','Y']
    )
)

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='Y')]


'?A(X,Y)'

,X,Y
0,1,1
1,1,2
2,2,3
3,2,2


In [ ]:
# union with different vars
test_session(
    """
        new B(int, int)
        new C(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        C(2, 2)
        C(1, 1)

        A(X, Y) <- B(X, Y)
        A(Z, W) <- C(Z, W)
        ?A(X, Y)
    """,
    pd.DataFrame(
        [[1,1],[1,2],[2,2],[2,3]],
        columns=['X','Y']
    )
)

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='Y')]


'?A(X,Y)'

,X,Y
0,1,1
1,1,2
2,2,3
3,2,2


In [ ]:
# test project
test_session(
    """
            new B(int, int)
            B(1, 1)
            B(1, 2)

            A(X) <- B(X, Y)
            ?A(X)
    """,
    pd.DataFrame(
        [[1]],
        columns=['X']
    )
)

adding relations for name='A' terms=[FreeVar(name='X')]


'?A(X)'

,X
0,1


In [ ]:
# add fact after rule
test_session(
    """
            new B(int, int)
            B(1, 1)
            A(X, Y) <- B(X, Y)
            B(1, 2)
            ?A(Z, W)
    """,
    pd.DataFrame(
        [[1,1],[1,2]],
        columns=['Z','W']
    )
)

adding relations for name='A' terms=[FreeVar(name='Z'), FreeVar(name='W')]


'?A(Z,W)'

,Z,W
0,1,1
1,1,2


In [ ]:
# test data types
test_session(
    """
            new B(int, str, span)
            B(1, "2", [1, 2))
            ?B(X, Y, Z)
    """,
    pd.DataFrame(
        [[1,"2",Span(1,2)]],
        columns=['X','Y','Z']
    )
)

adding relations for name='B' terms=[FreeVar(name='X'), FreeVar(name='Y'), FreeVar(name='Z')]


'?B(X,Y,Z)'

,X,Y,Z
0,1,2,"[1,2)"


In [ ]:
# join same relation
test_session(
    """
            new Parent(str, str)
            Parent("Sam", "Noah")
            Parent("Noah", "Austin")
            Parent("Austin", "Stephen")


            GrandParent(G, C) <- Parent(G, M), Parent(M, C)
            ?GrandParent(X, "Austin")
    """,
    pd.DataFrame(
        [['Sam']],
        columns=['X']
    )
)

adding relations for name='GrandParent' terms=[FreeVar(name='X'), 'Austin']


'?GrandParent(X,"Austin")'

,X
0,Sam


In [ ]:
# test_rule_with_constant
test_session(
"""
    new B(int, int)
    new C(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)
    C(2, 2)
    C(1, 1)
    A(X) <- B(1, X)
    ?A(X)
""",
pd.DataFrame([[1],[2]],columns=['X'])
)

adding relations for name='A' terms=[FreeVar(name='X')]


'?A(X)'

,X
0,1
1,2


In [ ]:
# test_rule_with_true_value
test_session(
"""
    new B(int, int)
    new C(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)
    C(2, 2)
    C(1, 1)

    A(X, Y) <- B(X, Y), C(1, 1)
    ?A(X, Y)
""",
pd.DataFrame([[1,1],[1,2],[2,3]],columns=['X','Y'])
)

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='Y')]


'?A(X,Y)'

,X,Y
0,1,1
1,1,2
2,2,3


In [ ]:
# test_rule_with_false_value
test_session(
"""
    new B(int, int)
    new C(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)
    C(2, 2)
    C(1, 1)

    A(X, Y) <- B(X, Y), C(0, 0)
    ?A(X, Y)
""",
pd.DataFrame([],columns=['X','Y'])
)

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='Y')]


'?A(X,Y)'

,X,Y


In [ ]:
# test_query_with_same_var
test_session(
[
    """
    new B(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)

    A(X, Y) <- B(X, Y)
    ?A(X,Y)
    """,
    """
    ?A(X, X)
"""
],[
    pd.DataFrame([[1,1],[1,2],[2,3]],columns=['X','Y']),
    pd.DataFrame([[1]],columns=['X']),
],
)

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='Y')]


'?A(X,Y)'

,X,Y
0,1,1
1,1,2
2,2,3



    ?A(X, X)

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='X')]
